In [ ]:
import mosdef_cassandra as mc
import numpy as np
import os
from pathlib import Path
# os.chdir(os.getcwd() + "/mc_bug_test")
# print(os.getcwd())

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.11/site-packages/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


/scratch365/mcarlozo/generalizedFF/mc_bug_test


In [3]:
def has_checkpoint(run_name):
    """Check whether there is a checkpoint for run_name."""
    fname = run_name + ".out.chk"
    return os.path.exists(fname)

def check_complete(run_name):
    """Check whether MoSDeF Cassandra simulation with run_name or its last restart has completed."""
    complete = False
    fname = run_name + ".out.log"
    loglist = list_with_restarts(fname)
    if not loglist:
        return complete
    with loglist[-1].open() as f:
        for line in f:
            if "Cassandra simulation complete" in line:
                complete = True
                break
    return complete

def list_with_restarts(fpath):
    """List fpath and its restart versions in order as pathlib Path objects."""
    fpath = Path(fpath)
    if not fpath.exists():
        return []
    parent = fpath.parent
    fname = fpath.name
    fnamesplit = fname.split(".out.")
    run_name = fnamesplit[0]
    suffix = fnamesplit[1]
    restarts = [
        Path(parent, f)
        for f in sorted(list(parent.glob(run_name + ".rst.*.out." + suffix)))
    ]
    restarts.insert(0, fpath)  # prepend fpath to list of restarts
    return restarts

def get_last_checkpoint(run_name):
    """Get name of last restart based on run_name."""
    fname = run_name + ".out.chk"
    return list_with_restarts(fname)[-1].name.split(".out.")[0]

In [7]:
prior_run = "gemc.eq"
cycles_done = 10000
equil_length = 500
for i in range(2):
    this_run = f"gemc.eq.rst.{i+1:03d}"
    print(this_run)
    if not has_checkpoint(this_run):
        print("No checkpoint found for", this_run)
        mc.restart(
            restart_from=prior_run,
            run_name=this_run,
            run_type="equilibration",
            total_run_length=cycles_done + equil_length,
        )
    elif not check_complete(this_run):
        print("Restarting from", get_last_checkpoint(this_run))
        mc.restart(
            restart_from=get_last_checkpoint(this_run),
        )
    cycles_done += equil_length
    prior_run = this_run

gemc.eq.rst.001
gemc.eq.rst.002
No checkpoint found for gemc.eq.rst.002
Using the following executables for Cassandra:
Python: /afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/bin/python
library_setup: /afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/bin/library_setup.py
Cassandra: /afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/bin/cassandra.exe


IndexError: string index out of range